# [순환신경망 실습: 다변량 시계열 분류 모델링]

### 1. 모듈 불러오기

In [ ]:
from google.colab import drive
drive.mount('content/gdrive/')
import os
os.chdir('/content/gdrove/My Drive/Day3/hands-on/3일차_RNN1/')

In [ ]:
import numpy as np
import pandas as pd

from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

import matplotlib.pyplot as plt


In [ ]:
seed = 190731
np.random.seed(seed)
torch.cuda.set_device(0)

### 2. 데이터 불러오기: Torch 데이터 준비하기(다채널시그널데이터)

In [ ]:
path = 'C:/Users/jiyoon/Dropbox/Project/한화딥러닝교육/코드자료/3일차_RNN2/'

X_train, y_train = torch.load(path+'Data/classification_multivariate_train_x'), torch.load(path+'Data/classification_multivariate_train_y')
X_valid, y_valid = torch.load(path+'Data/classification_multivariate_valid_x'), torch.load(path+'Data/classification_multivariate_valid_y')


- 시그널 데이터 살펴보기

In [ ]:
plt.figure(figsize=(16,8)) # 그래프 (가로길이, 세로길이)
plt.title('class: '+str(y_train[0].tolist()))
plt.plot(np.array(X_train[0,:,0])) # sensor 1
plt.plot(np.array(X_train[0,:,1])) # sensor 2
plt.plot(np.array(X_train[0,:,2])) # sensor 3
plt.plot(np.array(X_train[0,:,3])) # sensor 4
plt.plot(np.array(X_train[0,:,4])) # sensor 5
plt.plot(np.array(X_train[0,:,5])) # sensor 6
plt.plot(np.array(X_train[0,:,6])) # sensor 7
plt.plot(np.array(X_train[0,:,7])) # sensor 8
plt.plot(np.array(X_train[0,:,8])) # sensor 9
plt.plot(np.array(X_train[0,:,9])) # sensor 10
plt.show()

In [ ]:
for i in range(0,10):
    plt.figure(figsize=(16,8)) # 그래프 (가로길이, 세로길이)
    plt.title('class: '+str(y_train[i].tolist()))
    plt.plot(np.array(X_train[i,:,0])) # sensor 1
    plt.plot(np.array(X_train[i,:,1])) # sensor 2
    plt.plot(np.array(X_train[i,:,2])) # sensor 3
    plt.plot(np.array(X_train[i,:,3])) # sensor 4
    plt.plot(np.array(X_train[i,:,4])) # sensor 5
    plt.plot(np.array(X_train[i,:,5])) # sensor 6
    plt.plot(np.array(X_train[i,:,6])) # sensor 7
    plt.plot(np.array(X_train[i,:,7])) # sensor 8
    plt.plot(np.array(X_train[i,:,8])) # sensor 9
    plt.plot(np.array(X_train[i,:,9])) # sensor 10
    plt.show()

### 3. 데이터 전처리: Batch iterator 생성하기
- data_utils의 TensorDataset과 DataLoader를 사용해 batch iterator 생성하기

In [ ]:
train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)   

In [ ]:
batch_size = 128 # 사용자선택변수(hyperparameter)

In [ ]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size, shuffle=False)

print(f'Creating data loaders with batch size: {batch_size}')


### 3. RNN 모델 생성하기

In [ ]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return x

In [ ]:
input_dim = 10    
hidden_dim = 256
layer_dim = 3
output_dim = 9
seq_dim = 128
lr = 0.0005
n_epochs = 100

In [ ]:
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
#model = RNNClassifier(input_dim, hidden_dim, layer_dim, output_dim)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)


In [ ]:
print('Start model training')
for epoch in range(1, n_epochs + 1):
    # training part
    for i, (x_batch, y_batch) in enumerate(train_dl):
        model.train()       
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        opt.zero_grad()        
        out = model(x_batch)        
        loss = criterion(out, y_batch)
        loss.backward()        
        opt.step()
    
    # evaluating part
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in valid_dl:
        x_val, y_val = [t.cuda() for t in (x_val, y_val)]
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total
    
    print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. 평가정확도.: {acc:2.2%}')


In [ ]:
# RNN Epoch: 100. Loss: 1.3778. 평가정확도.: 45.14%
# LSTM Epoch: 100. Loss: 0.1447. 평가정확도.: 76.38%